In [1]:
#import library

import cv2 as cv #for image processing
import time #for current time control
import numpy as np
import os

In [2]:
#obtain camera meta-information

capture = cv.VideoCapture('./data/me/2022.1.10.12.26.14.raw.avi') #open camera
ret, frame = capture.read() #read camera's input
width = capture.get(cv.CAP_PROP_FRAME_WIDTH) #obtain width
height = capture.get(cv.CAP_PROP_FRAME_HEIGHT) #obtain height
fps = capture.get(cv.CAP_PROP_FPS) #obtain fps
#fps=8
print('width {0}, height {1}, fps {2}'.format(width, height, fps)) #print information

width 2592.0, height 1944.0, fps 30.0


In [3]:
class mouse:
    def __init__(self):
        self.position=None #
        self.kind=None #mother or baby
        self.ID=None #RFID
        self.center = None
    
    def update(self):
        pass

class enviroment:
    def __init__(self):
        self.tracking_center=[]
        
    

In [9]:
#main body. it describes all task for each frame.
number_mouse= 3
k_means_epoch = 5
start_time=time.time() 

flag = 1 #flag for record
flag_1= 1
flag_per = 0
delay = int(1000/fps) #delay based on fps
list_file=os.listdir('./data/me/')
count=0

for i in range(len(list_file)): #repeat infinitely
    
    capture = cv.VideoCapture('./data/me/'+str(list_file[i])) #open camera
    capture = cv.VideoCapture('./data/me/2022.1.11.0.26.3.raw.avi') #open camera
    _, frame_original_before = capture.read()
    frame_original_before=frame_original_before[220:1700,600:2100].copy()
    
    while 1==1: #read video
        ret, frame_original = capture.read() #read a frame
        frame_original = frame_original[220:1700,600:2100].copy()
        frame_original_after = (50+frame_original-frame_original_before)
        frame_original_after[0:230, 0:500]=0
        frame_range= cv.inRange(frame_original, (0,0,0), (60,60,60))
        frame_range[0:230, 0:500]=0
        frame_range[0:150, 1400:]=0
        frame_range[0:20]=0
        frame_range = cv.erode(frame_range, (5,5), iterations = 5)
        
        if flag_per == 3:  #for k-means algo
            flag_per=0
            
            temp_index = np.where(frame_original_after>100)
            acc_data = np.concatenate((np.reshape(temp_index[0],(-1,1)),np.reshape(temp_index[1],(-1,1))),axis=1)
            rgb_index = np.where(frame_range>254)
            rgb_data = np.concatenate((np.reshape(rgb_index[0],(-1,1)),np.reshape(rgb_index[1],(-1,1))),axis=1) 
            #total_data = np.concatenate((rgb_data,acc_data),axis=0)
            #total_data = acc_data  #stable rgb
            
            for k in range(k_means_epoch): #train center
                if flag ==1: 
                    flag=0
                    #print("d")
                    #print("d",len(total_data))
                    seed_x = np.random.randint(len(rgb_data),size=number_mouse)
                    seed_initial = rgb_data[0:3*number_mouse:3].copy()
                arrange_ = np.zeros((len(rgb_data),number_mouse))
                for j in range(number_mouse):
                    arrange_[:,j] = np.power((seed_initial[j,0]-rgb_data[:,0]),2) + np.power((seed_initial[j,1]-rgb_data[:,1]),2)
                arrange_group = np.argmin(arrange_,axis=1)
          
                for j in range(number_mouse):
                 
                    temp_arrange_index = np.where(j==arrange_group)
                    temp_len = len(temp_arrange_index[0])
                    temp_average_y = np.sum(rgb_data[temp_arrange_index][:,0])/temp_len
                    temp_average_x = np.sum(rgb_data[temp_arrange_index][:,1])/temp_len
                    seed_initial[j][0] = int(temp_average_y)
                    seed_initial[j][1] = int(temp_average_x)
                
                    #if frame_range[seed_initial[j,0],seed_initial[j,1]] != 255:
                    #    seed_x = np.random.randint(len(total_data),size=1)
                    #    seed_initial[j] = total_data[seed_x].copy()
                #print(len(acc_data))
            
            for k in range(k_means_epoch):
                if len(acc_data) >= 3 * number_mouse:
                    #print(len(acc_data))
                    if flag_1 ==1: 
                        flag_1=0
                        #print("d",len(total_data))
                        seed_x = np.random.randint(len(acc_data),size=number_mouse)
                        seed_initial_acc = acc_data[0:3*number_mouse:3].copy()
                        print("D",seed_initial_acc)
                    arrange_ = np.zeros((len(acc_data),number_mouse))
                    for j in range(number_mouse):
                        arrange_[:,j] = np.power((seed_initial_acc[j,0]-acc_data[:,0]),2) + np.power((seed_initial_acc[j,1]-acc_data[:,1]),2)
                    arrange_group = np.argmin(arrange_,axis=1)
                    print(arrange_group)
                    for j in range(number_mouse):
                        temp_arrange_index = np.where(j==arrange_group)
                        temp_len = len(temp_arrange_index[0])
                        #print(temp_len)
                        temp_average_y = np.sum(acc_data[temp_arrange_index][:,0])/temp_len
                        temp_average_x = np.sum(acc_data[temp_arrange_index][:,1])/temp_len
                        #print(temp_len)
                        seed_initial_acc[j][0] = int(temp_average_y)
                        seed_initial_acc[j][1] = int(temp_average_x)
            
            try:
                pass#seed_initial = 0.3*seed_initial + 0.7*seed_initial_acc
            except:
                pass

        frame_original_before = frame_original.copy()
        img_draw = frame_original_after.copy()
        if flag ==0:
            for j in range(number_mouse):
                cv.circle(frame_original, (seed_initial[j,-1],seed_initial[j,0]), 10, (0, 0, 255), -1)

        frame_resize=cv.resize(frame_original,(500,300))
        img_draw=cv.resize(img_draw,(500,300))
        frame_range=cv.resize(frame_range,(500,300))
        cv.imshow("mother",frame_resize)
        cv.imshow("fater",img_draw)
        cv.imshow("range",frame_range)
        
        key = cv.waitKey(1)  # if ESC key is pressed, then exit 
        if key ==27:
            break
            
        print(count)
        count+=1
        flag_per+=1
        
capture.release()  #terminate all the code.
cv.destroyAllWindows()




0
1
2
3
4
5
D [[ 69 711]
 [ 70 712]
 [ 71 712]]
[0 0 0 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
6
7
8
9
10
11
12
13
14
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2]


<ipython-input-9-6ded2ae1fe39>:85: RuntimeWarning: invalid value encountered in longlong_scalars
  temp_average_y = np.sum(acc_data[temp_arrange_index][:,0])/temp_len
<ipython-input-9-6ded2ae1fe39>:86: RuntimeWarning: invalid value encountered in longlong_scalars
  temp_average_x = np.sum(acc_data[temp_arrange_index][:,1])/temp_len


ValueError: cannot convert float NaN to integer